# Laboratorio 1

- Daniel Alfredo Rayo Roldan
- Gerardo Pineda Riveiro

## Repo:
https://github.com/DanielRasho/DS-Lab1.git

In [1]:
# pip install openpyxl instalar esto
import rdata
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Cargado de datos

In [10]:
xls = pd.read_excel("./data/preciosLimpio.xlsx", sheet_name=None)

resultados = {}

for nombre_hoja, df in xls.items():
    hoja_df = pd.read_excel("./data/preciosLimpio.xlsx", sheet_name=nombre_hoja, header=7, usecols="A,B,C,D,E,F,G,H")
    resultados[nombre_hoja] = hoja_df

In [11]:
for nombre_hoja in resultados:
    df = resultados[nombre_hoja]
    resultados[nombre_hoja] = df.iloc[1:].reset_index(drop=True)

In [17]:
resultados["2022"]

,Unnamed: 0,GTQ/USD,GTQ/GALON,GTQ/GALON.1,GTQ/GALON.2,GTQ/GALON.3,GTQ/CIL 25 LB,GTQ/LB,Año
0,2022-01-02,7.71912,28.69,27.91,24.51,17.28,122,4.88,2022
1,2022-01-03,7.71912,28.79,27.99,24.60,17.91,122,4.88,2022
2,2022-01-04,7.71745,28.79,27.99,24.60,17.91,122,4.88,2022
3,2022-01-05,7.71558,28.79,27.99,24.60,17.91,122,4.88,2022
4,2022-01-06,7.71823,28.79,27.99,24.60,17.91,122,4.88,2022
...,...,...,...,...,...,...,...,...,...
359,2022-12-27,7.84739,32.55,30.99,33.75,15.99,122,4.88,2022
360,2022-12-28,7.85028,32.55,30.99,33.75,15.99,122,4.88,2022
361,2022-12-29,7.85222,32.55,30.99,33.75,15.99,122,4.88,2022
362,2022-12-30,7.85222,32.55,30.99,33.75,15.99,122,4.88,2022


In [ ]:
df_final = pd.DataFrame()

for nombre_hoja, df in resultados.items():
    df["Año"] = nombre_hoja
    df_final = pd.concat([df_final, df], ignore_index=True)

In [13]:
hojas = pd.ExcelFile("./data/preciosLimpio.xlsx").sheet_names
print(hojas)

['2025', '2024', '2023', '2022', '2021']


In [14]:
df_final

,FECHA,Tipo de Cambio,Superior,Regular,Diesel,Bunker,Glp Cilindro 25Lbs.,Unnamed: 7,Año,Unnamed: 0,GTQ/USD,GTQ/GALON,GTQ/GALON.1,GTQ/GALON.2,GTQ/GALON.3,GTQ/CIL 25 LB,GTQ/LB
0,2025-01-01,7.70625,28.95,27.45,25.47,18.74,110,4.4,2025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-02,7.7074,28.95,27.45,25.47,18.74,110,4.4,2025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-03,7.70971,28.95,27.45,25.47,18.74,110,4.4,2025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-04,7.70971,28.95,27.45,25.47,18.74,110,4.4,2025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-05,7.70971,28.95,27.45,25.47,18.74,110,4.4,2025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,2021-12-27,7.71628,28.69,27.91,24.51,17.28,122.0,4.88
1649,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,2021-12-28,7.72447,28.69,27.91,24.51,17.28,122.0,4.88
1650,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,2021-12-29,7.71924,28.69,27.91,24.51,17.28,122.0,4.88
1651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,2021-12-30,7.71912,28.69,27.91,24.51,17.28,122.0,4.88


## Exploración de Datos

### Precios de Gasolina

In [ ]:
# Basic info
print(df.info())
print(df.describe())

# Check for missing values
print(df.isna().sum())

# Preview data
print(df.head())

In [ ]:
# Line plot of all fuel prices
df.plot(figsize=(14, 6), title="Fuel Prices Over Time")
plt.ylabel("Price")
plt.xlabel("Date")
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Example: use 'gasoline' price series
plt.figure(figsize=(10, 5))
plot_acf(df['gasoline'], lags=40)
plt.title("Autocorrelation Plot - Gasoline Prices")
plt.show()

### Precios de Diesel

### Precios de Gas Licuado

## Series temporales

### Precios de Gasolina